In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from quantulum3 import parser


In [2]:
url = 'https://cookingonabootstrap.com/2020/02/25/berry-buckwheat-pancake-recipe/'
res = requests.get(url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

In [3]:
matchers = ['(', ')']
matching = [line for line in text if any(chars in line for chars in matchers)]

In [4]:
for idx,line in enumerate(matching):
    bracket_start = line.find('(')
    bracket_end = line.find(')')
    slash_pos = line.find('/')
    if slash_pos != -1: #.find returns -1 on a no match
        str_price = line[bracket_start+1:slash_pos]
        #print(str_price)
        pound_pos = str_price.find('£')
        pence_pos = str_price.find('p')
        if pound_pos != -1:
            price = float(str_price[pound_pos+1:])
        if pence_pos != -1:
            price = float(str_price[:pence_pos])
            price = price/100
        print(price)
    #print(line[bracket_start+1:bracket_end])
    price = line[bracket_start+1:slash_pos]
    if idx >5:
        break



1.48
0.49
1.39
2.4
1.5
2.6


In [5]:
def get_units(line, slash_pos):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-5-8921ec2b7c1a>, line 2)

In [ ]:
unit = line[slash_pos+1]
unit

In [6]:
units = ['g', 'kg', 'l','ml', 'eggs']

In [8]:
parser.parse(line)

[Quantity(100, "Unit(name="gram", entity=Entity("mass"), uri=Gram)"),
 Quantity(26, "Unit(name="pint", entity=Entity("volume"), uri=Pint)"),
 Quantity(2.6, "Unit(name="pound sterling per kilogram", entity=Entity("unknown"), uri=None)")]

In [35]:
matching

['Buckwheat isn’t actually a grain – it’s from the same plant family as rhubarb, so ideal for gluten-free baking. (I’m not gluten-free, but my Mum is, and many of my readers are, so I do develop gf and coeliac-friendly recipes from time to time.)',
 '1 medium egg, 12p (£1.48/12 medium eggs, Asda)',
 '180ml whole milk, 9p (49p/1l, Asda)',
 '1 tsp baking powder, 3p (£1.39/170g, Dr Oetker at Asda)',
 '4 tbsp or 40g buckwheat flour, 10p (£2.40/kg, Doves Farm at Ocado)',
 '4 tbsp or 40g gluten-free flour, 6p (£1.50/kg, Asda)',
 '100g mixed berries, 26p (£2.60/kg, frozen at Asda)',
 "Jack Monroe is an award winning food writer and bestselling author. Books include A Girl Called Jack, A Year In 120 Recipes and Cooking On A Bootstrap. She has won the Fortnum & Mason Food and Drink award (ironically), the Observer Food Monthly Best Food Blog, Marie Claire 'Woman At The Top', Red Magazine's 'Red Hot Women', the YMCA Courage & Inspiration Award, the Woman Of The Year Entrepreneur award, the Women

In [9]:
parser.parse( '1 tsp baking powder, 3p (£1.39/170g, Dr Oetker at Asda)')

[Quantity(1, "Unit(name="teaspoon", entity=Entity("volume"), uri=Teaspoon)"),
 Quantity(3, "Unit(name="pint", entity=Entity("volume"), uri=Pint)")]

In [10]:
line2 =  '1 tsp baking powder, 3p (£1.39/170g, Dr Oetker at Asda)'

In [11]:
parser.parse(line2)

[Quantity(1, "Unit(name="teaspoon", entity=Entity("volume"), uri=Teaspoon)"),
 Quantity(3, "Unit(name="pint", entity=Entity("volume"), uri=Pint)")]

In [18]:
parser.parse(line2[31:])

[Quantity(170, "Unit(name="gram", entity=Entity("mass"), uri=Gram)")]

In [17]:
line2[28:]

'39/170g, Dr Oetker at Asda)'